
## Тест стандартной модели в задаче выделения пользователей, оставивших заявку в течение следующих 3 дней после анализируемого. База состоит из пользователей с приматченным телефоном.



In [43]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
import datetime
import sys

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "1g")
sc = SparkContext()
hc = HiveContext(sc)
sc.setCheckpointDir('checkpoint/')

In [70]:

pattern = '''
WITH t AS 
 (
   SELECT
   v.urlfr AS urlfr
   ,COUNT(*) AS cnt_positive   
  FROM
   (SELECT DISTINCT a.id FROM prod_features_liveinternet.user_action a WHERE  a.ymd between '#ymd1' and '#ymd3' and a.action_type = '#action_type') ta 
   inner join prod_features_liveinternet.visits v on v.id = ta.id and v.ymd='#ymd0'
  GROUP BY 
   v.urlfr
   ) 
  
INSERT OVERWRITE TABLE 
 prod_features_liveinternet.urlfr_tgt_cnt PARTITION (ymd='#ymd0', target='#action_type@tinkoff_action') 
SELECT 
 s.urlfr AS urlfr
 ,nvl(t.cnt_positive, 0) as cnt_positive
 ,s.visitors as cnt_total 
 ,current_timestamp() as load_dttm
FROM
 prod_features_liveinternet.urlfr_stat s 
 LEFT OUTER JOIN t ON t.urlfr = s.urlfr 
WHERE 
 s.ymd='#ymd0' AND 
 (s.visitors > 100 OR nvl(t.cnt_positive, 0) > 0 )
 
'''





In [71]:
import datetime
start_day = datetime.date(2016, 11, 02)
for i in range(0,55,1):
    ymd0 = start_day + datetime.timedelta(days = i)
    ymd1 = ymd0 + datetime.timedelta(days = 1)
    ymd3 = ymd0 + datetime.timedelta(days = 3)
    action_type = 'tinkoff_platinum_approved3_application'
    print(pattern.replace('#ymd0',str(ymd0)).replace('#ymd1',str(ymd1)).replace('#ymd3',str(ymd3)).replace('#action_type',action_type) + ';\n\n')
    #print('{}. Done for {}.'.format(datetime.datetime.now(),ymd0))


WITH t AS 
 (
   SELECT
   v.urlfr AS urlfr
   ,COUNT(*) AS cnt_positive   
  FROM
   (SELECT DISTINCT a.id FROM prod_features_liveinternet.user_action a WHERE  a.ymd between '2016-11-03' and '2016-11-05' and a.action_type = 'tinkoff_platinum_approved3_application') ta 
   inner join prod_features_liveinternet.visits v on v.id = ta.id and v.ymd='2016-11-02'
  GROUP BY 
   v.urlfr
   ) 
  
INSERT OVERWRITE TABLE 
 prod_features_liveinternet.urlfr_tgt_cnt PARTITION (ymd='2016-11-02', target='tinkoff_platinum_approved3_application@tinkoff_action') 
SELECT 
 s.urlfr AS urlfr
 ,nvl(t.cnt_positive, 0) as cnt_positive
 ,s.visitors as cnt_total 
 ,current_timestamp() as load_dttm
FROM
 prod_features_liveinternet.urlfr_stat s 
 LEFT OUTER JOIN t ON t.urlfr = s.urlfr 
WHERE 
 s.ymd='2016-11-02' AND 
 (s.visitors > 100 OR nvl(t.cnt_positive, 0) > 0 )
 
;



WITH t AS 
 (
   SELECT
   v.urlfr AS urlfr
   ,COUNT(*) AS cnt_positive   
  FROM
   (SELECT DISTINCT a.id FROM prod_features_liveinternet.

In [42]:
cumul_pattern = '''
INSERT OVERWRITE TABLE 
 user_kposminin.urlfr_tgt_cnt3 PARTITION (ymd='#ymd0', target='#action_type_avg_1month@tinkoff_action') 
SELECT 
 urlfr
 ,sum(cnt_positive)/#period as cnt_positive
 ,sum(cnt_total)/#period as cnt_total 
FROM
 user_kposminin.urlfr_tgt_cnt3 t 
WHERE 
  t.ymd between '#ymd_minus_period' and '#ymd0'
  and t.target = '#action_type@tinkoff_action'
group by
  urlfr

'''

period = 30
import datetime
start_day = datetime.date(2016, 11, 01)
for i in range(0,-21,-7):
    ymd0 = start_day + datetime.timedelta(days = i)
    ymd0_minus_period = ymd0 + datetime.timedelta(days = -period + 1)
    action_type = 'tinkoff_platinum_approved_application'
    print(cumul_pattern \
          .replace('#ymd0',str(ymd0)) \
          .replace('#ymd_minus_period',str(ymd0_minus_period)) \
          .replace('#action_type',action_type) \
          .replace('#period',str(period)) \
         + ';\n\n')
    #print('{}. Done for {}.'.format(datetime.datetime.now(),ymd0))


INSERT OVERWRITE TABLE 
 user_kposminin.urlfr_tgt_cnt3 PARTITION (ymd='2016-11-01', target='tinkoff_platinum_approved_application_avg_1month@tinkoff_action') 
SELECT 
 urlfr
 ,sum(cnt_positive)/30 as cnt_positive
 ,sum(cnt_total)/30 as cnt_total 
FROM
 user_kposminin.urlfr_tgt_cnt3 t 
WHERE 
  t.ymd between '2016-10-03' and '2016-11-01'
  and t.target = 'tinkoff_platinum_approved_application@tinkoff_action'
group by
  urlfr

;



INSERT OVERWRITE TABLE 
 user_kposminin.urlfr_tgt_cnt3 PARTITION (ymd='2016-10-25', target='tinkoff_platinum_approved_application_avg_1month@tinkoff_action') 
SELECT 
 urlfr
 ,sum(cnt_positive)/30 as cnt_positive
 ,sum(cnt_total)/30 as cnt_total 
FROM
 user_kposminin.urlfr_tgt_cnt3 t 
WHERE 
  t.ymd between '2016-09-26' and '2016-10-25'
  and t.target = 'tinkoff_platinum_approved_application@tinkoff_action'
group by
  urlfr

;



INSERT OVERWRITE TABLE 
 user_kposminin.urlfr_tgt_cnt3 PARTITION (ymd='2016-10-18', target='tinkoff_platinum_approved_application_a

In [ ]:

la_approved3_query = '''

create table user_kposminin.la_approved3_20161102 as
select
    m.phone_num
   ,count(distinct v.id) as id_cnt
   ,max(t.score) as score  
   ,v.ymd
from
    (select 
       uid_str as id,
       property_value as phone_num
     from
       prod_dds.md_uid_property 
     where
       property_cd = 'PHONE' and
       load_src = 'LI.02'
      ) m
    inner join prod_features_liveinternet.visits v on v.id = m.id
    left join (
     select urlfr, log((cnt_positive + 0.1)/(cnt_total - cnt_positive + 0.1)) as score
     from user_kposminin.urlfr_tgt_cnt3
     where target  = 'tinkoff_platinum_approved_application_avg_1month@tinkoff_action'      
       and ymd = '2016-10-25'
       and (cnt_total > 10000 or cnt_positive > 1)
    ) t on t.urlfr = v.urlfr
    left join (
      SELECT DISTINCT a.id,if(a.ymd = '2016-11-03',1,0) as first_day 
      FROM prod_features_liveinternet.user_action a 
      WHERE  a.ymd between '2016-11-03' and '2016-11-05' and a.action_type = 'tinkoff_platinum_approved_application'
    ) ta on ta.id = m.id
where 
  v.ymd = '2016-11-02'
  group by m.phone_num, v.ymd
;
'''

In [54]:
test = hc.sql('select label,first_day,nvl(score,-20) from user_kposminin.la_approved3_right_20161102').collect()

In [67]:
print('Positive 3 days {}.\nPositive 1 day  {}.\nTotal      {}.'.format(sum([e[0] for e in test]),sum([e[1] for e in test]),len(test)))

Positive 3 days 250.
Positive 1 day  100.
Total       23728052.


In [65]:
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
import sklearn.metrics as m
auc3 = m.roc_auc_score(y_true = [e[0] for e in test], y_score = [nvl(e[2],-30) for e in test])
auc1 = m.roc_auc_score(y_true = [e[1] for e in test], y_score = [nvl(e[2],-30) for e in test])
print('\nMax score ROC AUC for 3 days positive class:{}.\nMax score ROC AUC for 1 day  positive class:{}.\n'.format(
    auc3
    ,auc1
    ))



Max score ROC AUC for 3 days positive class:0.694570920307.
Max score ROC AUC for 1 day  positive class:0.704097528308.



In [55]:
data = sorted([[e[2],e[0]] for e in test], reverse = True)

def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for max score\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for max score

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	9         	3.60%     	2372      	0.37943%  	360.12    	-6.88712  
0.9998    	12        	4.80%     	4745      	0.25290%  	240.03    	-7.05889  
0.9997    	13        	5.20%     	7118      	0.18264%  	173.34    	-7.09279  
0.9995    	13        	5.20%     	11864     	0.10958%  	104.00    	-7.24178  
0.999     	13        	5.20%     	23728     	0.05479%  	52.00     	-7.64881  
0.998     	17        	6.80%     	47456     	0.03582%  	34.00     	-8.51158  
0.997     	19        	7.60%     	71184     	0.02669%  	25.33     	-9.14380  
0.995     	21        	8.40%     	118640    	0.01770%  	16.80     	-9.27110  
0.99      	29        	11.60%    	237280    	0.01222%  	11.60     	-9.49290  
0.98      	34        	13.60%    	474561    	0.00716%  	6.80      	-9.68035  
0.95      	47        	18.80%    	1186402   	0.00396%  	3.76      	-9.86077  
0.9       	79        	31.60%    	2372805   	0.003


# Вывод
### Общий AUC ROC практически равен, лишь чуть ниже, показаталя в задаче выделения полных заявок. Но в лифт таблице видно, что в самых верхних квантилях, которые нам и нужны, выделение даже более сильное (до $q_{0.9995}$).
### Таким образом, модель можно тестировать в бою.